In [2]:
import requests
from bs4 import BeautifulSoup

open_api_key = 'VwUNnuPS5treeoYbtM8R2Cw89HPvmvApECDi28RBhaY4z2eFUCx2WjEXUm1w%2B6iibQjSA5FxoeCMm3Pj5YDlhA%3D%3D'
params = '&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200315'

#url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?ServiceKey=' + open_api_key + params
#end_point = 'http://openapi.data.go.kr/openapi/service/rest/Covid19'
open_url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson?serviceKey=VwUNnuPS5treeoYbtM8R2Cw89HPvmvApECDi28RBhaY4z2eFUCx2WjEXUm1w%2B6iibQjSA5FxoeCMm3Pj5YDlhA%3D%3D&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20200414'


res = requests.get(open_url).text.encode('utf-8')
soup = BeautifulSoup(res, 'lxml-xml')

In [3]:
rows = soup.findAll('item')
rows[0]

<item><areaNm>기타</areaNm><areaNmCn>其他</areaNmCn><areaNmEn>Others</areaNmEn><createDt>2020-04-14 10:43:52.52</createDt><natDeathCnt>0</natDeathCnt><natDeathRate>0.0</natDeathRate><natDefCnt>1</natDefCnt><nationNm>생피에르미클롱</nationNm><nationNmCn>聖皮埃爾和密克隆群島</nationNmCn><nationNmEn>Saint Pierre and Miquelon</nationNmEn><seq>9521</seq><stdDay>2020년 04월 14일 09시</stdDay><updateDt/></item>

In [4]:
columns = rows[0].find_all()
columns

[<areaNm>기타</areaNm>,
 <areaNmCn>其他</areaNmCn>,
 <areaNmEn>Others</areaNmEn>,
 <createDt>2020-04-14 10:43:52.52</createDt>,
 <natDeathCnt>0</natDeathCnt>,
 <natDeathRate>0.0</natDeathRate>,
 <natDefCnt>1</natDefCnt>,
 <nationNm>생피에르미클롱</nationNm>,
 <nationNmCn>聖皮埃爾和密克隆群島</nationNmCn>,
 <nationNmEn>Saint Pierre and Miquelon</nationNmEn>,
 <seq>9521</seq>,
 <stdDay>2020년 04월 14일 09시</stdDay>,
 <updateDt/>]

In [5]:
import pandas as pd

rowlist = []
collist = []
tlist = []

for i in range(0, len(rows)):
    columns = rows[i].find_all()
    for j in range(0, len(columns)):
        if i == 0:
            tlist.append(columns[j].name)
        col_ = columns[j].text
        collist.append(col_)
    rowlist.append(collist)
    collist=[]
    
result_df = pd.DataFrame(rowlist, columns=tlist)
print('number of rows: ', len(result_df.index))
result_df.head(5)



number of rows:  5948


,areaNm,areaNmCn,areaNmEn,createDt,natDeathCnt,natDeathRate,natDefCnt,nationNm,nationNmCn,nationNmEn,seq,stdDay,updateDt
0,기타,其他,Others,2020-04-14 10:43:52.52,0,0.0,1,생피에르미클롱,聖皮埃爾和密克隆群島,Saint Pierre and Miquelon,9521,2020년 04월 14일 09시,
1,기타,其他,Others,2020-04-14 10:43:52.52,0,0.0,5,포클랜드제도,福克蘭群島,Falkland Islands,9520,2020년 04월 14일 09시,
2,기타,其他,Others,2020-04-14 10:43:52.52,0,0.0,3,카리브 네덜란드,加勒比荷蘭,Caribbean Netherlands,9519,2020년 04월 14일 09시,
3,기타,其他,Others,2020-04-14 10:43:52.52,2,18.181818181,11,북마리아나제도,北瑪麗安群島,Northern Mariana Is.,9518,2020년 04월 14일 09시,
4,기타,其他,Others,2020-04-14 10:43:52.52,0,0.0,3,영국령 버진아일랜드,英屬維爾京群島,British Virgin Islands,9517,2020년 04월 14일 09시,


In [6]:
df_covid = result_df[['areaNmEn', 'nationNmEn', 'createDt', 'natDeathCnt', 'natDeathRate', 'natDefCnt']]
df_covid.head(3)

,areaNmEn,nationNmEn,createDt,natDeathCnt,natDeathRate,natDefCnt
0,Others,Saint Pierre and Miquelon,2020-04-14 10:43:52.52,0,0.0,1
1,Others,Falkland Islands,2020-04-14 10:43:52.52,0,0.0,5
2,Others,Caribbean Netherlands,2020-04-14 10:43:52.52,0,0.0,3


In [11]:
#enable module: geopy to work in thie notebook
!pip install geopy

  Using cached geopy-2.1.0-py3-none-any.whl (112 kB)
  Using cached geographiclib-1.50-py3-none-any.whl (38 kB)


In [54]:
import numpy as np

def get_geocode(city): 
    try: 
        geoloc = Nominatim(user_agent="covid19") 
        return geoloc.geocode(city, timeout=None) 
    except GeocoderTimedOut: 
        return get_geocode(city)    

longitude = []
latitude = []


for i in (df_covid['nationNmEn']):
    if get_geocode(i) != None:
        location = get_geocode(i)
         
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)




In [56]:
df_covid['latitude'] = latitude
df_covid['longitude'] = longitude

<ipython-input-56-86c6a9a167c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['latitude'] = latitude
<ipython-input-56-86c6a9a167c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['longitude'] = longitude


In [60]:
df_covid = df_covid.dropna()
df_covid

,areaNmEn,nationNmEn,createDt,natDeathCnt,natDeathRate,natDefCnt,latitude,longitude
0,Others,Saint Pierre and Miquelon,2020-04-14 10:43:52.52,0,0.0,1,46.783247,-56.195159
1,Others,Falkland Islands,2020-04-14 10:43:52.52,0,0.0,5,-51.966642,-59.550039
2,Others,Caribbean Netherlands,2020-04-14 10:43:52.52,0,0.0,3,17.566000,-63.143239
3,Others,Northern Mariana Is.,2020-04-14 10:43:52.52,2,18.181818181,11,-28.755100,24.729985
4,Others,British Virgin Islands,2020-04-14 10:43:52.52,0,0.0,3,18.402439,-64.566164
...,...,...,...,...,...,...,...,...
5479,Asia,Japan,2020-03-19 10:52:16.16,31,3.3916849015,914,36.574844,139.239418
5480,Asia,Macau,2020-03-19 10:52:16.16,0,0.0,15,22.175760,113.551414
5481,Asia,Taiwan,2020-03-19 10:52:16.16,1,1.0,100,23.973937,120.982018
5482,Asia,HongKong,2020-03-19 10:52:16.16,4,2.0833333333,192,22.279328,114.162813


In [61]:
df_covid.to_excel('./covid_19.xlsx', index = False)

In [64]:
def test():
    a=[1]
    b=[2, 3]
    
    return a, b



<function test at 0x00000209820FBAF0>


In [72]:
df_korea = df_covid[df_covid['nationNmEn']=='South Korea']
df_korea

,areaNmEn,nationNmEn,createDt,natDeathCnt,natDeathRate,natDefCnt,latitude,longitude
215,Asia,South Korea,2020-04-14 10:43:50.50,222,2.1014767133,10564,36.638392,127.696119
431,Asia,South Korea,2020-04-13 10:36:36.36,217,2.0594096991,10537,36.638392,127.696119
647,Asia,South Korea,2020-04-12 10:29:43.43,214,2.0357686453,10512,36.638392,127.696119
862,Asia,South Korea,2020-04-11 10:58:18.18,211,2.0133587786,10480,36.638392,127.696119
1077,Asia,South Korea,2020-04-10 10:31:23.23,208,1.9904306220,10450,36.638392,127.696119
1292,Asia,South Korea,2020-04-09 10:06:10.10,204,1.9572100163,10423,36.638392,127.696119
1506,Asia,South Korea,2020-04-08 9:58:15.15,200,1.9260400616,10384,36.638392,127.696119
1719,Asia,South Korea,2020-04-07 10:22:27.27,192,1.8584841738,10331,36.638392,127.696119
1931,Asia,South Korea,2020-04-06 10:13:12.12,186,1.8086347724,10284,36.638392,127.696119
2142,Asia,South Korea,2020-04-05 10:08:07.07,177,1.7428121307,10156,36.638392,127.696119
